### Handling video files : extracting frames from a video

Given a video containing all the frames, we extract each individual frame to use as an RGB image.


In [1]:
from util.handle_files import * #Contains makedir() and list_files()
from functions.model_alignment import *
import cv2
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
def extract_frames_video(relative_filepath, extension):
    """
    input : relative_filepath (str), folder containing the video
            extension (str), extension of the video 
            
    Extracts frames from a video and saves them.
    Frames will be saved in a folder called frame in the same folder as the video,
    So far, assumes that you have only one video in the folder.
    ex:
        extension is 'mp4'
        video is contained in /directory/video/
        frames will be saved in /directory/video/frames/
    """
    #Getting the video filepath
    video = list_files(relative_filepath, extension)
    cap = cv2.VideoCapture(video)
    target_directory = makedir(relative_filepath+'/frames/')
    output_filename = target_directory+'frame_'
    print("Reading frames and saving to folder.")
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read() #Test whether a frame was read correctly
        if ret == False:
            break
        cv2.imwrite(output_filename+str(i)+'.jpg',frame)
        i+=1
    print("Done. A total of {} frames were saved.".format(i))
    cap.release()
    cv2.destroyAllWindows()

In [3]:
rel_path = '../worm_data/video/'

In [4]:
extract_frames_video(rel_path, '.avi')


Target directory doesn't exist. Creating directory under /Users/pedrobedmar/MEGA/EPFL/ML/project2/cs-433-project-2-worm_brains/code/../worm_data/video//frames/
Reading frames and saving to folder.
Done. A total of 1715 frames were saved.


### Handling .h5 files
The .h5 files are structured as such : 
```
├── file.h5
    ├── 0
    |   └── frame
    └── 1
    |   └── frame
   ... 
    └── ...
```
Where each number correspond to the frame number.
Within, we'll find a `frame` which contains the data for a given frame. They can be one channel (as is the case for the `file1`) or 3 channels (RGB, for `file2`). 

Each frame can be seen a stacks of 2D RGB images shape = (C, W, H). Thus, a `frame` is of shape **(C, W, H, Z)** where C is the channel, W and H the 2D dimensions (X and Y), and Z is the vertical Z-axis. 

To deal with the 3D component, we decided to use max project all the frames onto a 2D plane.

In [3]:
import h5py as h5
import numpy as np
import torch
import torchvision
from torchvision.utils import save_image
from torchvision.transforms import ToTensor, Resize, ToPILImage
from PIL import Image

PATH_h5 = '../worm_data/h5_data/'
#Loading files
file1 = h5.File(PATH_h5+'epfl3.h5')
file2 = h5.File(PATH_h5+'more_data.h5')

Having explored the data located in file1, we've found those single channels, 112x112 images to be unusable due to their low resolution.

Thus, only frames from file2 are extracted and used.

In [6]:
#Extracting frames from files
def extract_frames_h5(file, save_path, channel = 'rgb', resize=None):
    """
    reads an h5 file and extracts all the frames within
    Channel specify the channels to save
    can be 'red', 'green' or 'rgb'.
    paths are relative
    """
    if resize: #using torch.transforms
        res = Resize((resize,resize))
    maxvalues = [1543, 1760]
    #due to the structure, we keep only the files that are numeric
    filenames = [x for x in file.keys() if x.isnumeric()]
    filenames.sort(key=int) #Sort with key = int
    
    nb_channels = file['0/frame'].shape[0]
    w_h = file['0/frame'].shape[1]
    z = file['0/frame'].shape[3]
    channel = channel.lower()
    if ((nb_channels==1) and (channel=='rgb')):
        print("No frame extracted. Exiting.\nRGB specified, but only one color channel was found.")
        return
    
    PATH = makedir(save_path)
    PATH = makedir(save_path+channel+'_frames')
    print("Extracting {} frames.".format(channel))
    #For loop to extract all the frames
    for index, number in enumerate(filenames):
        name = number+'/frame'
        frame = file[name]
        #max projecting
        
        #creating RGB images with artificial blue "background" channel
        if nb_channels==2 and channel=='rgb':
            temp = torch.as_tensor(np.max(frame,axis=3),dtype=torch.float32) 
            blue = torch.full((1,512,512),0, dtype=torch.float32)
            image_tensor = torch.cat((temp,blue))
            #Normalizing to range [0,1]
            image_tensor[0,...].div_(maxvalues[0])
            image_tensor[1,...].div_(maxvalues[1])
        
        if channel!='rgb':
            index = 0
            if channel == 'green':
                index = 1
            image_tensor = torch.as_tensor(np.max(frame,axis=3),dtype=torch.float32)
            if nb_channels!=1:
                image_tensor[index,...].div_(maxvalues[index])
                
            image_tensor = image_tensor[index:index+1,...]
    
        if resize:
            #Using this convoluted way due to some errors within pytorch
            #with how it handles tensors/PILImage for Resize.
            to_pili_image = ToPILImage()
            image_tensor = to_pili_image(image_tensor)
            image_tensor = ToTensor()(res(image_tensor))
            
        save_image(image_tensor, PATH+'/frame_'+number+'.jpg')
            
    print("Done. A total of {} frames were saved.".format(len(filenames)))

In [7]:
sizes = [240, 512]
colors = ['rgb', 'red', 'green']

#Extract frames in different ways and sizes.
for size in sizes:
    for col in colors:
        if size == 240:
            path = PATH_h5+str(size)+'/'
        elif size == 512:
            path = PATH_h5+str(size)+'/'
        extract_frames_h5(file2, path, channel=col, resize=size)

Extracting rgb frames.
Done. A total of 572 frames were saved.
Extracting red frames.
Done. A total of 572 frames were saved.
Extracting green frames.
Done. A total of 572 frames were saved.
Extracting rgb frames.
Done. A total of 572 frames were saved.
Extracting red frames.
Done. A total of 572 frames were saved.
Extracting green frames.
Done. A total of 572 frames were saved.
